In [1]:
%matplotlib widget
import pandas as pd
import numpy as np
import csv
import os
import hdbscan
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'ipympl'

In [ ]:
def  genereate_df_from_embeddings(file_path, species):
    size = 0
    data = np.matrix(np.arange(424))
    entries = os.scandir(file_path)#'./output/Caryothraustes')
    for entry in entries:
        try:
            if (entry.is_file()):
                path = entry.path
                f = open(path, "r", encoding="cp1252", errors='ignore')
                embedding = f.read().strip().split("\n")
                name = entry.name
                for em in embedding:
                    em = em.strip().split(",")
                    em_times = em.pop(0).split("\t")
                    em.insert(0, em_times[2])
                    em.insert(0, em_times[1])
                    em.insert(0, em_times[0])
                    em = [float(i) for i in em]
                    
                    em.insert(0, species)
                    em.insert(0, name.split(".")[0])

                    #if (data == None):
                    #    data = np.matrix(np.arange(len(em)))
                    #    size = len(em)
                    data = np.append(data, [em],axis=0)
        except Exception as e:
            print("issue with ", entry.name, e)
    data = np.delete(data, 0, 0)
    return pd.DataFrame(data = data, columns=np.append(["name", "species", "start", "end"], np.arange(420)))

Caryothraustes = genereate_df_from_embeddings('./output/Caryothraustes', "Caryothraustes canadensis")

In [ ]:
file_paths = ['./output/Caryothraustes', './output/Lipaugus', './output/Anthus crenatus','./output/Hirundo aethiopica', './output/Mulleripicus pulverulentus']
species = ["Caryothraustes canadensis",  "Lipaugus vociferans", "Anthus crenatus", "Hirundo aethiopica", "Mulleripicus pulverulentus"]
df_list = []

for i in range(0, len(species)):
    df = genereate_df_from_embeddings(file_paths[i], species[i])
    df_list.append(df)
df_list

In [ ]:
embeddings_df = pd.concat(df_list)
embeddings_df

In [ ]:
#embeddings.to_csv("./embeddings-test.csv")

In [ ]:
def get_species_label(animal):
    return species.index(animal)

In [ ]:
cluster_data = embeddings_df.get( map(str, np.append(range(0,420), "species"))).assign(species=embeddings_df.get("species").apply(get_species_label))
cluster_data

In [ ]:
from sklearn.mixture import GaussianMixture
#clusterer = hdbscan.HDBSCAN()
#data = embeddings_df.get( map(str, range(0,420)))
#clusterer.fit(embeddings_df.get( map(str, range(0,420))))
gm= GaussianMixture(n_components=len(species), random_state=0).fit_predict(cluster_data)
#gm.labels_
gm

In [ ]:
embeddings_df = embeddings_df.assign(cluster=gm)#gm.labels_)
embeddings_df[embeddings_df.get("cluster") == -1]

In [ ]:
pca = PCA(n_components = 3) # Default n_components = min(n_samples, n_features)
X_train_pc = pca.fit_transform(embeddings_df.get( map(str, range(0,420))))

pd.DataFrame(pca.components_.T)
X_train_pc

In [ ]:
np.unique(gm)#clusterer.labels_)

In [ ]:
X_train_pc

In [ ]:
   
#clusterer.labels_

pca_embeddings = pd.DataFrame(data={"cluster": gm, "species": list(embeddings_df.get("species").apply(get_species_label))})
pca_embeddings = pd.concat([pca_embeddings, pd.DataFrame(X_train_pc)], axis=1)
pca_embeddings.columns = ["cluster", "species", "x", "y", "z"]
#pca_embeddings = pca_embeddings.assign(species=embeddings_df.get("species"))
pca_embeddings

In [ ]:
embeddings_to_plot = pca_embeddings[pca_embeddings.get("cluster") == -1]
plot = plt.scatter(embeddings_to_plot.get("x"), embeddings_to_plot.get("y"), embeddings_to_plot.get("z"), c=embeddings_to_plot.get("species"))

In [ ]:
embeddings_to_plot = pca_embeddings[pca_embeddings.get("cluster") != -1]
plot = plt.scatter(embeddings_to_plot.get("x"), embeddings_to_plot.get("y"), c=embeddings_to_plot.get("cluster"))

In [ ]:
embeddings_to_plot = pca_embeddings
plot = plt.scatter(embeddings_to_plot.get("x"), embeddings_to_plot.get("y"), c=embeddings_to_plot.get("cluster"))

In [ ]:
embeddings_to_plot = pca_embeddings
plot = plt.scatter(embeddings_to_plot.get("x"), embeddings_to_plot.get("y"), c=embeddings_to_plot.get("species"))

In [ ]:
embeddings_to_plot.get("species")

In [ ]:
embeddings_to_plot = pca_embeddings
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
plot = ax.scatter(embeddings_to_plot.get("x"), embeddings_to_plot.get("y"),embeddings_to_plot.get("z"), c=embeddings_to_plot.get("species"))
plot;

In [ ]:
embeddings_to_plot = pca_embeddings
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
plot = ax.scatter(embeddings_to_plot.get("x"), embeddings_to_plot.get("y"),embeddings_to_plot.get("z"), c=embeddings_to_plot.get("cluster"))
plot;